#ch07-How to Develop a DCGAN for Grayscale Handwritten Digits
- [ch07-How to Develop a DCGAN for Grayscale Handwritten Digits](https://machinelearningmastery.com/how-to-develop-a-generative-adversarial-network-for-an-mnist-handwritten-digits-from-scratch-in-keras/) by [Jason Brownlee](https://machinelearningmastery.com/author/jasonb/) on Sept 1, 2020 in [Generative Adversarial Networks](https://machinelearningmastery.com/category/generative-adversarial-networks/)
- Customized by Ivan HP Lin (WIP)